In [ ]:
import nest
import os
import anndata
import scipy
import sklearn.metrics
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import squidpy as sq
import networkx as nx
from pathlib import Path
import time
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.stats import pearsonr


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import rc

import leidenalg as la
import igraph as ig
from scipy.sparse import coo_array
from scipy.spatial.distance import cdist
from scipy.spatial import KDTree
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.colors as colors

import warnings
warnings.filterwarnings("ignore")

In [ ]:
available_datasets = ["V1_Mouse_Brain_Sagittal_Anterior", "V1_Mouse_Brain_Sagittal_Posterior", "seqfish", "merfish",
                     "slideseq", "V1_Breast_Cancer_Block_A_Section_1"]
dataset = "seqfish"

In [ ]:
image_save_dir = os.path.expanduser(f"images/{dataset}/")
from pathlib import Path
Path(image_save_dir).mkdir(parents=True, exist_ok=True)
nest.plot.set_dataset_plot_parameters(dataset)
def save_fig(fig, name):
    fig.savefig(os.path.join(image_save_dir, name), dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
adata = nest.data.get_data(dataset)


In [ ]:
nest.spatial_smoothing(adata)

In [ ]:
nest.get_min_samples_from_density(adata, eps=0.12, density=0.1)

In [ ]:
nest.compute_gene_hotspots(adata, eps=0.12, min_size=25, min_samples=5, log=True)
#nest.interaction_hotspots(adata, eps=0.20, min_size=25, min_samples=15, method="permutation", log=False, save_activity=True, sig_threshold=0.92)

In [ ]:
#nest.plot.spatial(adata, color=("Dll1", "Notch1"), color_map="Blues", use_raw=False)

In [ ]:
#nest.plot.hotspots(adata, color="DLL1_NOTCH1")

In [ ]:
nest.coexpression_hotspots(adata, threshold=0.30, min_size=15, min_genes=3, resolution=1.0, cutoff=0.5)

In [ ]:
nest.compute_multi_boundaries(adata, 10, 0.00001)

In [ ]:
nest.plot.multi_hotspots(adata)

In [ ]:
#nest.plot.all_coex_hotspots(adata, figsize=(10, 20))

In [ ]:
nest.plot.spatial(adata, color="class")

In [ ]:
nest.plot.spatial(adata, color=("Notch1", "Dll1", "Dll3", "DLL1_NOTCH1"), color_map="Blues", use_raw=False)

In [ ]:
cc = nest.methods.CellChat()
res = cc.run(adata, group_by="class")
cc.cellchat_score(adata, interaction="Dll1  - Notch1")

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
nest.plot.spatial(adata, color="cellchat_score", title="CellChat (nonspatial)",
                  color_map="Blues", ax=ax, frameon=False)

In [ ]:
%matplotlib inline
rc('font',**{'family':'serif','serif':['Arial'], 'size':5})
# ligand, receptor, spatial activity, hotspots, cellchat score
fig, axs = plt.subplots(1, 6, figsize=(6.5, 3))
keys = ["Dll1", "Notch1", "DLL1_NOTCH1", "cellchat_score"]
ax_ids = [0,1,2,4]
nest.plot.spatial(adata, color="class", frameon=False, show=False, title="", 
                  groups=['Forebrain/Midbrain/Hindbrain', 'Presomitic mesoderm', 'Spinal cord'],
                  legend_loc='right margin', na_in_legend=False, ax=axs[5])
for key, idx in zip(keys, ax_ids):
    nest.plot.spatial(adata, color=key, ax=axs[idx], color_map="Blues", frameon=False, show=False, title="",
                     legend_loc=False, use_raw=False)
    axs[idx].collections[0].colorbar.remove()
nest.plot.hotspots(adata, color="DLL1_NOTCH1", ax=axs[3], show=False, title="", frameon=False)
fig.savefig(os.path.join(image_save_dir, f'comparison.pdf'), dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
nest.plot.spatial(adata, color="class", frameon=False, show=False, title="", 
                  groups=['Forebrain/Midbrain/Hindbrain', 'Presomitic mesoderm', 'Spinal cord'],
                  legend_loc="on data", na_in_legend=False)

In [ ]:
# List of all interactions identified in the dataset
adata.uns['interactions']

In [ ]:
fig, ax = plt.subplots(figsize=(1, 1))
nest.plot.hotspots(adata, color="WNT5A_FZD2", labels=False, frameon=False, title="", ax=ax, show=False)
save_fig(fig, "wnt_summary.pdf")

In [ ]:
nest.plot.spatial(adata, color=("Wnt5a", "Fzd2", "WNT5A_FZD2"), color_map="Blues", use_raw=False)